In [1]:
import pandas as pd
import json
import numpy as np
from time import sleep
import time
from tqdm import tqdm
import requests
from pydantic import BaseModel
url = open("local_models_path.txt", "r").read()

In [2]:
# Load the test datasets
test_en = pd.read_json("../../datasets/copa-en-test.jsonl", lines=True)
test_sl = pd.read_json("../../datasets/copa-sl-test.jsonl", lines=True)
test_hr_ckm = pd.read_json("../../datasets/copa-hr-ckm-test.jsonl", lines=True)
test_hr = pd.read_json("../../datasets/copa-hr-test.jsonl", lines=True)
test_mk = pd.read_json("../../datasets/copa-mk-test.jsonl", lines=True)
test_sl_cer = pd.read_json("../../datasets/copa-sl-cer-test.jsonl", lines=True)
test_sr = pd.read_json("../../datasets/copa-sr-test.jsonl", lines=True)
test_sr_tor = pd.read_json("../../datasets/copa-sr-tor-test.jsonl", lines=True)

print(test_en.shape, test_sl.shape, test_hr.shape, test_hr_ckm.shape, test_mk.shape, test_sl_cer.shape, test_sr.shape, test_sr_tor.shape)

(500, 5) (500, 5) (500, 7) (500, 5) (500, 7) (500, 5) (500, 7) (500, 5)


In [ ]:
tests = ["copa-en","copa-sl", "copa-hr", "copa-hr-ckm", "copa-mk", "copa-sl-cer", "copa-sr", "copa-sr-tor"]

In [4]:
def run_local_model(model, prompt, url=url):

	class ReponseStructure(BaseModel):
		answer: int

	data = {
	    "model": model,
	    "prompt": prompt,
	    "stream": False,
	    "temperature": 0,
	    "format": ReponseStructure.model_json_schema()
	}

	headers = {"Content-Type": "application/json",}
	response = requests.post(url, json=data, headers=headers)

	return response.json()["response"]

In [5]:
models = ["gemma3:27b", "llama3.3:latest", "qwen3:32b", "deepseek-r1:14b"]

In [ ]:
def predict_gpt(df_test_name, gpt_model):

	df_path = f"../../datasets/{df_test_name}-test.jsonl"

	responses = []

	start_time = time.time()
	for line in open(df_path):
		entry=json.loads(line)
		if df_test_name != "copa-en":
			prompt= 'You will be given a task. The task definition is in English, but the task itself is in another language. Here is the task!\nGiven the premise "'+entry['premise']+'",'
			if entry['question']=='cause':
				prompt+=' and that we are looking for the cause of this premise,'
			else:
				prompt+=' and that we are looking for the result of this premise, '
			prompt+=f"""which hypothesis is more plausible?\nHypothesis 1: "{entry['choice1']}".\nHypothesis 2: "{entry['choice2']}".
					
			### Output format
				Return a valid JSON dictionary with the following key: 'answer' and a value should be an integer -- either 1 (if hypothesis 1 is more plausible) or 2 (if hypothesis 2 is more plausible).
			"""
		elif df_test_name == "copa-en":
			prompt= 'You will be given a task. The task definition is in English, as is the task itself. Here is the task!\nGiven the premise "'+entry['premise']+'",'
			if entry['question']=='cause':
				prompt+=' and that we are looking for the cause of this premise,'
			else:
				prompt+=' and that we are looking for the result of this premise,'
			prompt+=f"""which hypothesis is more plausible?\nHypothesis 1: "{entry['choice1']}".\nHypothesis 2: "{entry['choice2']}".
					
			### Output format
				Return a valid JSON dictionary with the following key: 'answer' and a value should be an integer -- either 1 (if hypothesis 1 is more plausible) or 2 (if hypothesis 2 is more plausible).
			"""

		if gpt_model == "GaMS-27B":
			gpt_model_path = "hf.co/mradermacher/GaMS-27B-i1-GGUF"
		else:
			gpt_model_path = gpt_model

		initial_response= run_local_model(gpt_model_path, prompt, url=url)

		response = initial_response.replace("\n", "")
		response = response.replace("\t", "")

		# Convert the string into a dictionary
		response = json.loads(response)

		# Get out a label
		try:
			# The true labels are 0 or 1, so you have to change the answer by substracting 1 from it.
			predicted = response["answer"]-1
			responses.append(predicted)
		# add a possibility of something going wrong
		except:
			predicted = initial_response
			print("error with extracting a label:")
			print(initial_response)
			responses.append(initial_response)

	end_time = time.time()
	elapsed_time_min = end_time-start_time

	print(f"Prediction finished. It took {elapsed_time_min/60} min for 500 instances - {elapsed_time_min/500} s per instance.")

	# Create a json with results

	current_results = {
		"system": gpt_model,
		"predictions": [
			{
			"train": "NA (zero-shot)",
			"test": "{}".format(df_test_name),
			"predictions": responses,
			},
		]
		}

	# Save the results as a new json
	with open("submissions/submission-{}-{}.json".format(gpt_model, df_test_name), "w") as file:
		json.dump(current_results, file)

	print("Classification with {} on {} finished.".format(gpt_model, df_test_name))

In [8]:
# First, check that everything works well by evaluating the models on English
for model in models:
	print(model)
	predict_gpt("copa-en", model)

gemma3:27b
Prediction finished. It took 5.618045790990194 min for 500 instances - 0.6741654949188233 s per instance.
Classification with gemma3:27b on copa-en finished.
llama3.3:latest
Prediction finished. It took 5.147024305661519 min for 500 instances - 0.6176429166793823 s per instance.
Classification with llama3.3:latest on copa-en finished.
qwen3:32b
Prediction finished. It took 3.066311510403951 min for 500 instances - 0.3679573812484741 s per instance.
Classification with qwen3:32b on copa-en finished.
deepseek-r1:14b


KeyboardInterrupt: 

In [ ]:
tests = ["copa-en","copa-sl", "copa-hr", "copa-hr-ckm", "copa-mk", "copa-sl-cer", "copa-sr", "copa-sr-tor"]

In [ ]:
# Now, evaluate the models on all other datasets, without DeepSeek, as it takes the longest
for test in tests[1:]:
	for model in ["gemma3:27b", "llama3.3:latest", "qwen3:32b"]:
		print(model)
		predict_gpt(test, model)


gemma3:27b


In [ ]:
# Add the DeepSeek
for test in tests:
	predict_gpt(test, "deepseek-r1:14b")

In [ ]:
import requests


url = 'http://kt-gpu5.ijs.si:11435/api/generate'
data = {
    "model": "GaMS-27B-Instruct", #"hf.co/mradermacher/GaMS-27B-i1-GGUF-i1-Q4_K_M",
    "prompt": "Explain what machine learning is.",
    "stream": False
}


headers = {"Content-Type": "application/json"}
response = requests.post(url, json=data, headers=headers)

result = response.json()

result

{'error': "model 'GaMS-27B-Instruct' not found"}

In [ ]:
# Add GaMS:
for test in tests:
	predict_gpt(test, "GaMS-27B")